In [1]:
import pandas as pd
import numpy as np

In [2]:
items = pd.read_csv('items_export.csv')
items.rename(columns={'Origin [Recipe]':'origin', 'Recipe ingredient':'ingredient', 'Name [Food]':'name'}, inplace=True)
items = items[items.origin != 'mama' ]
items['ingredient'] = items['ingredient'].str.replace("’","'")
    

In [3]:
food_matches = items[['ingredient']].copy()
food_matches['food'] = ""
food_matches['quantity'] = ""
food_matches['measurement'] = ""

In [4]:
foods = pd.read_csv('/Users/vincentsalamand/Downloads/food_export.csv')
foods = foods.name

In [96]:
from spacy.lang.fr import French
import spacy
nlp = French()

doc = list(nlp.pipe(items.ingredient))

In [6]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = list(nlp.tokenizer.pipe(foods))
matcher.add("FOOD_PATTERN", None, *patterns)


In [62]:
from textblob import TextBlob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

food_parser = []
for line in doc:
    matches = matcher(line)
    elements = []    
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text)
        food_parser.append(max(elements, key=len).lower())
    else:
        text_blob_object = TextBlob(line.text)
        singular_line = ' '.join(text_blob_object.words.singularize())
        matches = matcher(tokenizer(singular_line))
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = tokenizer(singular_line)[start:end]
                elements.append(span.text)
            food_parser.append(max(elements, key=len).lower())  
        else:
            plural_line = ' '.join(text_blob_object.words.pluralize())
            matches = matcher(tokenizer(plural_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(plural_line)[start:end]
                    elements.append(span.text)
                food_parser.append(max(elements, key=len).lower())
            else:    
                ratios = process.extractOne(line.text.lower(), foods)
                if len(ratios) > 0:
                    food_parser.append(ratios[0])
              
             
len(parser)

5115

In [8]:
pd.set_option('display.max_rows', 500)
food_matches['food'] = parser
food_matches

,ingredient,food,quantity,measurement
0,2.0 filets huile d'olive,huile d'olive,,
1,1.0 steak haché,steak haché,,
2,70.0 grammes pâtes,pâtes,,
3,500 g de semoule moyenne,semoule,,
4,500 g de tomate environ (ébouillantées pour ot...,tomate,,
5,1 concombre (épluché et en gros morceaux),concombre,,
6,5 oignon (épeluchés et coupé en 4),oignon,,
7,5 branche de persil et de menthe,persil,,
8,Citron et autant d'huile,citron,,
9,Poivre,poivre,,


In [10]:
food_matches[food_matches.food.astype(str) == "[]"]

,ingredient,food,quantity,measurement


In [11]:
len(food_matches[food_matches.food.astype(str) == "[]"])

0

In [12]:
measurements = ["gr.", "gr", "g", "g.", "gramme",
                "kg.", "kg", "kilogramme",
                "mg.", "mg", "miligramme",
                "ml.", "ml", "mililitre",
                "cl.", "cl", "centilitre",
                "l.", "l", "litre",
                "c. à soupe", "cuillère à soupe", "cs", "cas", "càs", "c. à table", "cuillère à table",
                "c. à café", "cuillère à café", "cc", "cac", "càc", "c. à thé", "cuillère à thé",
                "tasse", "bol", "verre", "filet", "zeste",
                "noisette", "noix", "pincée", "pointe", "poignée", 
                "feuille", "branche", "gousse", "tranche", "cube",
                "boîte", "barquette", "pot", "bâtonnet", "boule", "rouleau"    
]

In [13]:
measurement_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
measurement_patterns = list(nlp.tokenizer.pipe(measurements))
measurement_matcher.add("MEASUREMENT_PATTERN", None, *measurement_patterns)

In [15]:
import re

measurement_parser = []

for line in doc:
    line = tokenizer(re.sub('[0-9]', ' ', line.text))
    matches = measurement_matcher(line)
    elements = [] 
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text)
        measurement_parser.append(elements[0].lower())
    else:
        text_blob_object = TextBlob(line.text)
        singular_line = ' '.join(text_blob_object.words.singularize())
        matches = measurement_matcher(tokenizer(singular_line))
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = tokenizer(singular_line)[start:end]
                elements.append(span.text)
            measurement_parser.append(elements[0].lower())  
        else:
            plural_line = ' '.join(text_blob_object.words.pluralize())
            matches = measurement_matcher(tokenizer(plural_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(plural_line)[start:end]
                    elements.append(span.text)
                measurement_parser.append(elements[0].lower())
            else:    
                 measurement_parser.append("")

In [16]:
food_matches['measurement'] = measurement_parser
food_matches

,ingredient,food,quantity,measurement
0,2.0 filets huile d'olive,huile d'olive,,filet
1,1.0 steak haché,steak haché,,
2,70.0 grammes pâtes,pâtes,,gramme
3,500 g de semoule moyenne,semoule,,g
4,500 g de tomate environ (ébouillantées pour ot...,tomate,,g
5,1 concombre (épluché et en gros morceaux),concombre,,
6,5 oignon (épeluchés et coupé en 4),oignon,,
7,5 branche de persil et de menthe,persil,,branche
8,Citron et autant d'huile,citron,,
9,Poivre,poivre,,


In [94]:
quantities = ["½","1½", "⅓", "¼", "un demi", "une demi","un et demi","tiers", "quart","zero","deux","trois","quatre",
              "cinq", "six", "sept","huit","neuf","dix","onze","douze","treize","quatorze","quinze","seize","dix-sept",
              "dix-huit","dix-neuf","vingt","trente","quarante","cinquante","soixante","soixante-dix","quatre-vingt",
              "quatre-vingt-dix","dizaine","une dizaine","douzaine","une douzaine","demi-douzaine","une demi-douzaine","vingtaine","une vingtaine", "trentaine","quarantaine",
              "cinquantaine","centaine","une centaine", "cent", "un", "une"]

In [90]:
quantity_phrasematcher = PhraseMatcher(nlp.vocab, attr="LOWER")
quantity_patterns = list(nlp.tokenizer.pipe(quantities))
quantity_phrasematcher.add("MEASUREMENT_PATTERN", None, *quantity_patterns)

In [91]:
from spacy.matcher import Matcher
quantity_matcher = Matcher(nlp.vocab)
quantity_pattern = [{"LIKE_NUM": True}]
quantity_matcher.add("NUM_PATTERN", None, quantity_pattern)



In [92]:
import re
char_list = [i for i in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèàâêïù▪️"]


quantity_parser = []

for line in doc:
    matches = quantity_phrasematcher(line)
    elements = [] 
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text)
        quantity_parser.append(elements[0])
    else:
        matches = quantity_matcher(line)
        elements = [] 
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = line[start:end]
                elements.append(span.text)
            quantity_parser.append(elements[0])
        else:               
            strip_line = re.sub("|".join(char_list), "", line.text)
            matches = quantity_matcher(tokenizer(strip_line))
            elements = [] 
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(strip_line)[start:end]
                    elements.append(span.text)
                quantity_parser.append(elements[0])
            else:
                quantity_parser.append("")

                

In [87]:
import re
char_list = [i for i in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèàâêïù▪️"]
zizi = food_matches[food_matches.index == 8367].ingredient.values[0]
zizi = re.sub("|".join(char_list), "", zizi)
toto = tokenizer(re.sub("|".join(char_list), "", zizi))
print(zizi)
matches = quantity_matcher(toto) 
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = toto[start:end]  # The matched span
    print(span.text)
len(matches)

1½  (375 ) '


0

In [93]:
food_matches['quantity'] = quantity_parser
food_matches

,ingredient,food,quantity,measurement
0,2.0 filets huile d'olive,huile d'olive,2.0,filet
1,1.0 steak haché,steak haché,1.0,
2,70.0 grammes pâtes,pâtes,70.0,gramme
3,500 g de semoule moyenne,semoule,500,g
4,500 g de tomate environ (ébouillantées pour ot...,tomate,500,g
5,1 concombre (épluché et en gros morceaux),concombre,1,
6,5 oignon (épeluchés et coupé en 4),oignon,5,
7,5 branche de persil et de menthe,persil,5,branche
8,Citron et autant d'huile,citron,,
9,Poivre,poivre,,
